In [1]:
import os
import io

import cv2 as cv
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from PIL import Image as pil_image
from IPython.html import widgets

from ipywidgets import interact, interact_manual
from IPython.display import display, Image, clear_output

/usr/local/lib/python3.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


# Utilities

In [2]:
def read_folder(folder, ext):
    images = []
    for filename in sorted(os.listdir(folder)):
        if filename.endswith(ext):
            path = os.path.join(folder, filename)
            images.append(np.array(pil_image.open(path)))
    return images

def display_frames_sequence(frames):
    def exec(i):
        a = np.uint8(frames[i])
        f = io.StringIO()

        imgByteArr = io.BytesIO()
        pil_image.fromarray(a, 'RGB').save(imgByteArr, format='PNG')
        display(Image(data=imgByteArr.getvalue()))
        
    widgets.interact(exec, i=widgets.IntSlider(min=0, max=len(frames) - 1, step=1, value=0))
    
def read_frames_dataset(path, sep=',', ext='.jpg'):
    frames = read_folder(os.path.join(path, 'img/'), ext)
    df = pd.read_csv(os.path.join(path, 'groundtruth_rect.txt'), names=['x', 'y', 'w', 'h'], sep=sep)
    return frames, df

def enreach_single_frame_with_circles(frame, points, color=(255, 0, 0)):
    frame = np.copy(frame)
    for point in points:
        cv.circle(frame, (int(point[0][0]), int(point[0][1])), 3, color, -1)
    return frame

def write_video(path, frames):
    fourcc = cv.VideoWriter_fourcc(*'MP4V')
    h, w = frames[0].shape[:2]
    out = cv.VideoWriter(path + ".mp4", fourcc, 24, (w, h)) 
    for frame in frames:
        out.write(cv.cvtColor(frame, cv.COLOR_BGR2RGB))
    cv.destroyAllWindows()
    out.release()

# Load and configure data

[Datasets source](http://cvlab.hanyang.ac.kr/tracker_benchmark/datasets.html)

In [3]:
names = {
    "bike": './datasets/MountainBike/',
    "bolt": './datasets/Bolt/', 
    "car": './datasets/CarScale/'
}

In [4]:
bike, bike_df = read_frames_dataset(names["bike"])
bolt, bolt_df = read_frames_dataset(names["bolt"])
car, car_df = read_frames_dataset(names["car"], sep='\t')

# example
bike_df.head()

,x,y,w,h
0,319,185,67,56
1,316,183,67,56
2,315,182,66,56
3,313,181,66,56
4,311,181,66,55


# Lucas-Kanade

In [5]:
class LucasKanade:
    
    TERMINATION_CRITERIA = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1)
    
    def __init__(self, init_frame, roi, use_pyramidal_extension=False):
        self.max_level = 0 if use_pyramidal_extension else 8
        self.init_frame = cv.cvtColor(init_frame, cv.COLOR_BGR2GRAY)
        self.template = self.init_frame[roi["y"]:roi["y"] + roi["h"], roi["x"]:roi["x"] + roi["w"]]
        self.roi = roi
        self.gftt = None
        self.points = None
        
    def fit(self):
        gftt = cv.goodFeaturesToTrack(
            self.template, 
            mask=None, 
            maxCorners=30, 
            qualityLevel=0.02, 
            minDistance=6,
            blockSize=6,
        )
        self.points = np.add(gftt, np.array([self.roi["x"], self.roi["y"]]))
        self.points = self.points.astype(np.float32)
        return self.points
       
    def predict(self, frame):
        frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        self.points, _, _ = cv.calcOpticalFlowPyrLK(
            self.init_frame,
            frame,
            self.points,
            None,
            winSize=(25, 25),
            maxLevel=self.max_level,
            criteria=LucasKanade.TERMINATION_CRITERIA
        )
        
        self.init_frame = frame
        return self.points
    
    @staticmethod
    def run(frames, df, use_pyramidal_extension=True): 
        init = df.iloc[0]
        lk = LucasKanade(frames[0], init, use_pyramidal_extension=use_pyramidal_extension)
        
        points = lk.fit()
        result = [points]
        enreached_frames = [enreach_single_frame_with_circles(frames[0], points)]
        
        for frame in frames[1:]:
            points = lk.predict(frame)
            
            result.append(points)
            enreached_frames.append(enreach_single_frame_with_circles(frame, points))
            
        return enreached_frames

# "Bike" dataset

In [6]:
enreached_frames_bike = LucasKanade.run(bike, bike_df)
display_frames_sequence(enreached_frames_bike)
write_video("./results/lk_bike", enreached_frames_bike)

interactive(children=(IntSlider(value=0, description='i', max=227), Output()), _dom_classes=('widget-interact'…

# "Bolt" dataset

In [7]:
enreached_frames_bolt = LucasKanade.run(bolt, bolt_df)
display_frames_sequence(enreached_frames_bolt)
write_video("./results/lk_bolt", enreached_frames_bolt)

interactive(children=(IntSlider(value=0, description='i', max=349), Output()), _dom_classes=('widget-interact'…

# "CarScale" dataset

In [8]:
enreached_frames_car = LucasKanade.run(car, car_df)
display_frames_sequence(enreached_frames_car)
write_video("./results/lk_car", enreached_frames_car)

interactive(children=(IntSlider(value=0, description='i', max=251), Output()), _dom_classes=('widget-interact'…

# Conclusions

**Lucas-Kanade method** works well when object has no destortions on foreground and background has homogenious structure, otherwise, points are being lost during the tracking process.